In [4]:
import glob
import sys
sys.path.append('..')
from Clustering_Functions import *
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from pyclustering.cluster.kmedians import kmedians as pyclust_kmedians
from pyclustering.utils.metric import distance_metric, type_metric

In [47]:
# My own code for kmedians from scratch

def Manhattan_dist(A,B):
    return sum(np.abs(A-B))

def my_kmedians(election, k=2, proxy='Borda', borda_style='pes', n_init=5, max_steps = 100, return_centroids=False):
    """
    Returns the clustering obtained by applying the k-medians algorithm to the proxies of the ballots.

    Args:
        election : dictionary matching ballots with weights.
        k : the number of clusters desired.
        proxy : choice of {'Borda', 'HH'} for Borda or head-to-head proxy vectors.
        borda_style : choice of {'pes', 'avg'}, which is passed to Borda_vector (only if proxy == 'Borda') 
        n_init : the algorithm runs n_init independent times with different starting centers each time, and outputs the clustering that has the best score from all the runs.
        return_centroids : set to True if you want it to also return the centroids of the returned clustering.

    Returns:
        if return_centroids == False: returns a clustering (list of elections).
        if return_centroids == True: returns a tuple (clustering, centroids).
    """

    num_cands = max([item for ranking in election.keys() for item in ranking])

    # create a matrix, X, whose rows are the ballots (repeated as many times as the ballot was cast) 
    # and a dictionary matching each ballot type with its first corresponding row in the matrix
    # and a reverse dictionary to match each row number of the matrix with a ballot
    X = []
    ballot_to_row = dict()
    row_to_ballot = dict()
    counter = 0
    for ballot, weight in election.items():
        ballot_to_row[ballot]=counter
        for _ in range(weight):
            if proxy=='Borda':
                X.append(Borda_vector(ballot, num_cands=num_cands, borda_style=borda_style))
            else:
                X.append(HH_proxy(ballot,num_cands=num_cands))
            row_to_ballot[counter]=ballot
            counter +=1
            
    # describe X as a dataframe
    dimension = len(X[0])
    df = pd.DataFrame(X)
    df['assignment']=0
    df['best_assignment']=0
    best_score = float('inf') #infinity
    best_centers = None

    for _ in range(n_init):
        # randomly select k ballot proxies as the initial medians
        initial_medians = []
        initial_ballots = []
        for count in range(k):
            ballot = random.choice(list(election.keys()))
            while ballot in initial_ballots: # re-do if duplication                    
                ballot = random.choice(list(election.keys()))
            initial_ballots.append(ballot)
            initial_medians.append(X[ballot_to_row[ballot]])    
    
        medians = initial_medians
        diff_count = 1
        step_count = 0
        while diff_count>0 and step_count<max_steps:
            step_count +=1
            diff_count = 0
            # find new clusters
            score = 0
            for ballot, weight in election.items():
                row_num = ballot_to_row[ballot]
                data_point = X[row_num]
                #data_point = df.loc[row_num][:-2]
                distances = [Manhattan_dist(data_point,center) for center in medians]
                score += weight*min(distances)
                assignment = np.argmin(distances)
                if df.loc[row_num,'assignment'] != assignment:
                    diff_count +=1
                    df.loc[row_num,'assignment'] = assignment

            # find new centers
            medians = [np.median(df[df['assignment']==a].loc[:,0:dimension-1], axis = 0) for a in range(k)]

        if score < best_score:
            print(score)
            df['best_assignment']=df['assignment']
            best_centers = medians

    # convert the best assignment into a list of dictionaries
    C = [dict() for _ in range(k)]
    for ballot, weight in election.items():
        assignment=df.loc[ballot_to_row[ballot]]['best_assignment']
        C[int(assignment)][ballot]=weight

    if return_centroids:
        return C, best_centers
    else:
        return C  

In [48]:
C, centers_test = my_kmedians(election, k=2, proxy='Borda', borda_style='pes', n_init=5, return_centroids=True)

c:\Users\ktapp\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])


114664.0
128164.0
128164.0
128164.0
114664.0


c:\Users\ktapp\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])


In [49]:
centers_test

[array([4., 0., 0., 0., 0., 1., 0.]), array([0., 3., 5., 4., 4., 0., 4.])]

In [75]:
def kmedians(election, k=2, proxy='Borda', borda_style='pes', n_init=50, return_centroids=False):
    """
    Returns the clustering obtained by applying the k-medians algorithm to the proxies of the ballots.

    Args:
        election : dictionary matching ballots with weights.
        k : the number of clusters desired.
        proxy : choice of {'Borda', 'HH'} for Borda or head-to-head proxy vectors.
        borda_style : choice of {'pes', 'avg'}, which is passed to Borda_vector (only if proxy == 'Borda') 
        n_init : the algorithm runs n_init independent times with different starting centers each time, and outputs the clustering that has the best score from all the runs.
        return_centroids : set to True if you want it to also return the centroids of the returned clustering.

    Returns:
        if return_centroids == False: returns a clustering (list of elections).
        if return_centroids == True: returns a tuple (clustering, centroids).
        (the centroids live in the proxy space and are the component-wise medians. They do not necessarily correspond to actual ballots)
    """
    num_cands = max([item for ranking in election.keys() for item in ranking])

    # create a matrix whose rows are the ballots (repeated as many times as the ballot was cast) 
    # and a dictionary matching each ballot type with its first corresponding row in the matrix
    # and a reverse dictionary to match each row number of the matrix with a ballot
    X = []
    ballot_to_row = dict()
    row_to_ballot = dict()
    counter = 0
    for ballot, weight in election.items():
        ballot_to_row[ballot]=counter
        for _ in range(weight):
            if proxy=='Borda':
                X.append(Borda_vector(ballot, num_cands=num_cands, borda_style=borda_style))
            else:
                X.append(HH_proxy(ballot,num_cands=num_cands))
            row_to_ballot[counter]=ballot
            counter +=1
            
    metric = distance_metric(type_metric.MANHATTAN)
    best_score = float('inf') #infinity
    best_centers = None
    for _ in range(n_init):
        # randomly select k ballot proxies as the initial medians
        initial_medians = []
        initial_ballots = []
        for count in range(k):
            ballot = random.choice(list(election.keys()))
            while ballot in initial_ballots: # re-do if duplication                    
                ballot = random.choice(list(election.keys()))
            initial_ballots.append(ballot)
            initial_medians.append(X[ballot_to_row[ballot]])

        model = pyclust_kmedians(X, ccore=True, initial_medians = initial_medians, 
                                metric = metric)
        model.process()
        subsets = model.get_clusters()
        centers = model.get_medians() 

        # pyclustering doesn't output the model's score, so we need to waste time computing it ourselves here.
        score = 0
        if len(centers)<k: # in the rare chance that the centers are the same
            score = float('inf')
        else:
            for ballot, weight in election.items():
                row_num = ballot_to_row[ballot]
                data_point = X[row_num]
                distances = [metric(data_point,center) for center in centers]
                score += (1/2)*weight*min(distances)
        if score<best_score:
            #print(score)
            best_score = score
            best_subsets = subsets
            best_centers = centers
            
    # convert best_subsets information into a list of dictionaries
    C = [dict() for _ in range(k)]
    for clust in range(k):
        for row_num in best_subsets[clust]:
            ballot = row_to_ballot[row_num]
            C[clust][ballot]=election[ballot]
    if return_centroids:
        return C, best_centers
    else:
        return C

In [8]:
# The Pentland Hills election that's studied in the paper
filename = '../scot-elex/7_cands/edinburgh_2017_ward2.csv'
num_cands, election, cand_names, ward = csv_parse(filename)
parties = party_abrevs(cand_names)

In [76]:
for count in range(10):
    C, centers = kmedians(election, k=2, proxy='Borda', borda_style='pes', n_init=20, return_centroids=True)
    centers = {0:centers[0], 1:centers[1]}
    score, _ = Cluster_from_given_centers(election, centers, proxy_type='BP', centers_live_in_proxy_space=True)
    if score == 52509:
        print(centers)


{0: [5.0, 4.0, 0.0, 0.0, 0.0, 6.0, 0.0], 1: [0.0, 0.0, 6.0, 0.0, 5.0, 0.0, 4.0]}
{0: [5.0, 4.0, 0.0, 0.0, 0.0, 6.0, 0.0], 1: [0.0, 0.0, 6.0, 0.0, 5.0, 0.0, 4.0]}
{0: [5.0, 4.0, 0.0, 0.0, 0.0, 6.0, 0.0], 1: [0.0, 0.0, 6.0, 0.0, 5.0, 0.0, 4.0]}
{0: [5.0, 4.0, 0.0, 0.0, 0.0, 6.0, 0.0], 1: [0.0, 0.0, 6.0, 0.0, 5.0, 0.0, 4.0]}
{0: [0.0, 0.0, 6.0, 0.0, 5.0, 0.0, 4.0], 1: [5.0, 4.0, 0.0, 0.0, 0.0, 6.0, 0.0]}
{0: [5.0, 4.0, 0.0, 0.0, 0.0, 6.0, 0.0], 1: [0.0, 0.0, 6.0, 0.0, 5.0, 0.0, 4.0]}
{0: [5.0, 4.0, 0.0, 0.0, 0.0, 6.0, 0.0], 1: [0.0, 0.0, 6.0, 0.0, 5.0, 0.0, 4.0]}
{0: [5.0, 4.0, 0.0, 0.0, 0.0, 6.0, 0.0], 1: [0.0, 0.0, 6.0, 0.0, 5.0, 0.0, 4.0]}
{0: [5.0, 4.0, 0.0, 0.0, 0.0, 6.0, 0.0], 1: [0.0, 0.0, 6.0, 0.0, 5.0, 0.0, 4.0]}
{0: [5.0, 4.0, 0.0, 0.0, 0.0, 6.0, 0.0], 1: [0.0, 0.0, 6.0, 0.0, 5.0, 0.0, 4.0]}


In [77]:
David_centers = {0:[5,0,0,0,0,5,0], 1:[0,0,5,2,5,0,4]}
score, _ = Cluster_from_given_centers(election, centers, proxy_type='BP', centers_live_in_proxy_space=True)
print(score)

52509.0


In [71]:
centers_test = {0:centers_test[0], 1:centers_test[1]}
test_score, _ = Cluster_from_given_centers(election, centers_test, proxy_type='BP', centers_live_in_proxy_space=True)
print(test_score)

52509.0


In [13]:
C, centers = kmedians(election, k=2, proxy='Borda', borda_style='pes', n_init=50, return_centroids=True)

In [14]:
centers

[[5.0, 4.0, 0.0, 0.0, 0.0, 6.0, 0.0], [0.0, 0.0, 6.0, 0.0, 5.0, 0.0, 4.0]]

In [15]:
C, centers = kmedians(election, k=2, proxy='Borda', borda_style='pes', n_init=200, return_centroids=True)

In [20]:
BP_centers = {i:centers[i] for i in range(2)}
BP_centers

{0: [5.0, 4.0, 0.0, 0.0, 0.0, 6.0, 0.0],
 1: [0.0, 0.0, 6.0, 0.0, 5.0, 0.0, 4.0]}

In [22]:
BP_David = {0:[5,0,0,0,0,5,0], 1:[0,0,5,2,5,0,4]}
BP_David

{0: [5, 0, 0, 0, 0, 5, 0], 1: [0, 0, 5, 2, 5, 0, 4]}

In [31]:
BP_score, C = Cluster_from_given_centers(election, BP_centers, proxy_type='BP', centers_live_in_proxy_space=True)
print(BP_score)

BP_David_score, C = Cluster_from_given_centers(election, BP_David, proxy_type='BP', centers_live_in_proxy_space=True)
print(BP_score)

52509.0
52509.0


In [23]:
def Cluster_from_given_centers(election, centers, proxy_type, order=1, centers_live_in_proxy_space = False):
    """
    Given the centers, cluster the ballots according to their closest center
    and return the summed distance and the clustering.
    ARGS:
        election: dictionary mapping ballots to weights
        centers: dictionary mapping center index to ballot or ballot proxy
        proxy_type: one of {'BA', 'BP', 'HH'} for Borda average, Borda pessimistic, or HH proxy
        order: the choice of p for L^p distance.  Use 1 for Manhattan, 2 for Euclidean.
        centers_live_in_proxy_space: True if the centers are in proxy space, False if they are in ballot space
    RETURNS:
        (summed distance, clustering)
        summed distance = sum of L^p distances from each ballot to its closest center
        clustering = dictionary mapping index to cluster.
    """
    num_cands = max([item for ranking in election.keys() for item in ranking])
    k = len(centers)

    C = {i:dict() for i in range(k)} # initialize clustering
    running_sum = 0
    for ballot, weight in election.items():
        if centers_live_in_proxy_space:
            if proxy_type == 'BA':
                ballot_proxy = Borda_vector(ballot, num_cands, borda_style='avg')
            elif proxy_type == 'BP':
                ballot_proxy = Borda_vector(ballot, num_cands, borda_style='pes')
            elif proxy_type == 'HH':
                ballot_proxy = HH_proxy(ballot, num_cands)
            dists = [(1/2)*np.linalg.norm(ballot_proxy - centers[i],ord=order) for i in centers.keys()]
        else:
            if proxy_type == 'BA':
                dists = [Borda_dist(ballot, centers[i], num_cands, borda_style='avg', order=order) for i in centers.keys()]
            elif proxy_type == 'BP':
                dists = [Borda_dist(ballot, centers[i], num_cands, borda_style='pes', order=order) for i in centers.keys()]
            elif proxy_type == 'HH':
                dists = [HH_dist(ballot, centers[i], num_cands, order=order) for i in centers.keys()]

        running_sum += weight*np.min(dists)
        clusts = [x for x in range(k) if dists[x]==np.min(dists)] # multi-valued argmin
        for clust in clusts:
            C[clust][ballot]=weight/len(clusts)
    return running_sum, C

In [40]:
C, HH_centers = kmedians(election, k=2, proxy='HH', n_init=10000, return_centroids=True)
HH_centers = {0:HH_centers[0], 1:HH_centers[1]}

HH_centers_IP = {0:[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, -1.0, 0.0, -1.0, 0.0, 1.0],
                 1:[0.0, -1.0, -1.0, -1.0, 0.0, -1.0, -1.0, 0.0, -1.0, 0.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 0.0, 1.0, 1.0, -1.0]}

print(HH_centers)

HH_score, C = Cluster_from_given_centers(election, HH_centers, proxy_type='HH', centers_live_in_proxy_space=True)
print(HH_score)

HH_David_score, C = Cluster_from_given_centers(election, HH_centers_IP, proxy_type='HH', centers_live_in_proxy_space=True)
print(HH_David_score)

{0: [0.0, -1.0, -1.0, -1.0, 0.0, 0.0, -1.0, -1.0, -1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], 1: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, -1.0, 0.0, -1.0, 0.0, -1.0, 0.0, -1.0, 0.0, 1.0, -1.0, 1.0, -1.0, 0.0, 1.0]}
52786.0
48681.0


In [38]:
HH_score, C = Cluster_from_given_centers(election, HH_centers, proxy_type='HH', centers_live_in_proxy_space=True)
print(HH_score)

HH_David_score, C = Cluster_from_given_centers(election, HH_centers_IP, proxy_type='HH', centers_live_in_proxy_space=True)
print(HH_David_score)

52786.0
48681.0
